# 项目信息

In [1]:
import pandas as pd
import numpy as np
import dateutil as dt

## 数据加载

基础文件说明:

- ./data/01_administrative_divisions.xlsx: 行政区划代码表, 目前只记录了与园区有关的行政区域
  - code: 行政区划代码
  - level: 级别. 0-中国 1-省份 2-地市 3-区县
  - pid: 父节点 id
- ./data/02_organization.xlsx: 组织机构表
  - level: 级别. 0-事业部 1-分公司 2-项目部
  - territory_id: 归属地 id. 关联 administrative_divisions.xlsx 文件中的 id
  - pid: 父节点 id
- ./data/03_industry_parks.xlsx: 园区信息/项目部信息, 这里的信息实际是上面"组织机构表"中级别为'2'的机构信息
  - org_id: 组织机构 id. 关联 organization.xlsx 文件中的 id
  - status: 园区状态. [筹开期, 在建期, 待建期, 成熟期, 培育期]
  - management_type: 管理类型. [垂直管理, 专业赋能]
  - territory_id: 归属地 id. 关联 administrative_divisions.xlsx 文件中的 id. 单独设置该字段而不采用 "所属组织(org_id)" 的归属地, 是因为项目的归属地与项目所属组织的归属地会不同.
  - product_deparment: 产品系. [网谷系, 意库系, 智慧城及其他, 创库系]
- ./data/04_projects.xlsx: 项目
  - ORGAN_ID: 租赁平台中的项目 ID
  - 归属机构级别: 对应"组织机构表"中的'level'字段, 0-事业部 1-分公司 2-项目部
  - 事业部/分公司/项目部: 这三个字段是为了便于编辑查看, 不应直接使用, 而应根据'归属机构ID'和'归属机构级别'从"组织机构表"中关联相关机构信息.


In [64]:
# load administrative divisions
df_ad = pd.read_excel('./data/01_administrative_divisions.xlsx', converters={'pid': int})
# load organization
df_org = pd.read_excel('./data/02_organization.xlsx', converters={'pid': int})
# load industry park
df_park = pd.read_excel('./data/03_industry_parks.xlsx', usecols='A:E')
# load project
df_project = pd.read_excel('./data/04_projects.xlsx', usecols='A:G')
# load project 包含机构 1 2 3 级
df_project_full = pd.read_excel('./data/05_project_full.xlsx')
# dict for df
dic = {
    'ad': df_ad,
    'org': df_org,
    'park': df_park,
    'project': df_project,
    'project_full': df_project_full
}

{k: v.columns for k, v in dic.items()}

{'ad': Index(['id', 'code', 'name', 'level', 'pid'], dtype='object'),
 'org': Index(['id', 'name', 'pid', 'level', 'territory_id'], dtype='object'),
 'park': Index(['id', 'org_id', 'status', 'management_type', 'product_department'], dtype='object'),
 'project': Index(['id', 'ORGAN_ID', '项目名称', '财务账套名称', '项目业态', '归属机构ID', '归属机构级别'], dtype='object'),
 'project_full': Index(['ORGAN_ID', '项目名称', '财务账套名称', 'level_1', 'level_2', 'level_3', '省', '市',
        '区'],
       dtype='object')}

In [12]:
df_org_ad = pd.merge(df_org, df_ad, left_on='territory_id', right_on='id', suffixes=('_org', '_ad'))
df_org_ad

,id_org,name_org,pid_org,level_org,territory_id,id_ad,code,name_ad,level_ad,pid_ad
0,1,产业园区事业部,NaN,0,3,3,440300,深圳市,2,2
1,5,深圳公司,1,1,3,3,440300,深圳市,2,2
2,9,南油平方,1,1,3,3,440300,深圳市,2,2
3,10,文化产业公司,1,1,3,3,440300,深圳市,2,2
4,11,园区运营中心,1,1,3,3,440300,深圳市,2,2
5,24,福海意库,5,2,3,3,440300,深圳市,2,2
6,25,福永意库,5,2,3,3,440300,深圳市,2,2
7,26,光明科技园,5,2,3,3,440300,深圳市,2,2
8,27,南海意库,5,2,3,3,440300,深圳市,2,2
9,29,蛇口网谷,5,2,3,3,440300,深圳市,2,2


In [16]:
df_project_rel = pd.merge(df_project, df_org_ad, left_on='归属机构ID', right_on='id_org')
df_project_rel

,id,ORGAN_ID,项目名称,财务账套名称,项目业态,归属机构ID,归属机构级别,id_org,name_org,pid_org,level_org,territory_id,id_ad,code,name_ad,level_ad,pid_ad
0,1,1427244,招商庐州意库,合肥德瀚产业园发展有限公司,园区,20,2,20,合肥庐州意库,3,2,8,8,340100,合肥市,2,7
1,2,1424215,芯云谷一期华为数据中心,福建招商云谷开发有限公司,园区,30,2,30,漳州芯云谷,5,2,19,19,350600,漳州市,2,18
2,3,1426209,芯云谷一期孵化器,福建招商云谷开发有限公司,园区,30,2,30,漳州芯云谷,5,2,19,19,350600,漳州市,2,18
3,4,1426211,芯云谷一期高层总部,福建招商云谷开发有限公司,园区,30,2,30,漳州芯云谷,5,2,19,19,350600,漳州市,2,18
4,5,1426213,芯云谷一期人才公寓,福建招商云谷开发有限公司,公寓,30,2,30,漳州芯云谷,5,2,19,19,350600,漳州市,2,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1412212,金山意库,重庆招商金山意库商业管理有限公司,园区,34,2,34,金山意库,7,2,4,4,500000,重庆市,1,1
96,97,1434200,九龙意库A栋,重庆招商意库商业管理有限公司,园区,35,2,35,九龙意库,7,2,4,4,500000,重庆市,1,1
97,98,1434202,九龙意库B栋,重庆招商意库商业管理有限公司,园区,35,2,35,九龙意库,7,2,4,4,500000,重庆市,1,1
98,99,1434204,九龙意库C栋,重庆招商意库商业管理有限公司,园区,35,2,35,九龙意库,7,2,4,4,500000,重庆市,1,1


In [27]:
pd.merge(df_project, df_org_ad[df_org_ad['level_org'] == 0], how='left', left_on='归属机构ID', right_on='id_org')

,id,ORGAN_ID,项目名称,财务账套名称,项目业态,归属机构ID,归属机构级别,id_org,name_org,pid_org,level_org,territory_id,id_ad,code,name_ad,level_ad,pid_ad
0,1,1427244,招商庐州意库,合肥德瀚产业园发展有限公司,园区,20,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1424215,芯云谷一期华为数据中心,福建招商云谷开发有限公司,园区,30,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1426209,芯云谷一期孵化器,福建招商云谷开发有限公司,园区,30,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1426211,芯云谷一期高层总部,福建招商云谷开发有限公司,园区,30,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1426213,芯云谷一期人才公寓,福建招商云谷开发有限公司,公寓,30,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,1412212,金山意库,重庆招商金山意库商业管理有限公司,园区,34,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,97,1434200,九龙意库A栋,重庆招商意库商业管理有限公司,园区,35,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,98,1434202,九龙意库B栋,重庆招商意库商业管理有限公司,园区,35,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,99,1434204,九龙意库C栋,重庆招商意库商业管理有限公司,园区,35,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_org

,id,name,pid,level,territory_id
0,1,产业园区事业部,NaN,0,3
1,2,杭州公司,1,1,13
2,3,南京公司,1,1,10
3,4,青岛公司,1,1,17
4,5,深圳公司,1,1,3
5,6,武汉公司,1,1,15
6,7,重庆公司,1,1,4
7,8,番禺科技园,1,1,17
8,9,南油平方,1,1,3
9,10,文化产业公司,1,1,3


In [54]:
df_org_full = df_org.copy()
df_org_full['level_0'] = 1

df_org_full

,id,name,pid,level,territory_id,level_0
0,1,产业园区事业部,NaN,0,3,1
1,2,杭州公司,1,1,13,1
2,3,南京公司,1,1,10,1
3,4,青岛公司,1,1,17,1
4,5,深圳公司,1,1,3,1
5,6,武汉公司,1,1,15,1
6,7,重庆公司,1,1,4,1
7,8,番禺科技园,1,1,17,1
8,9,南油平方,1,1,3,1
9,10,文化产业公司,1,1,3,1


In [60]:
df_project_full = pd.read_excel('./data/05_project_full.xlsx')
df_project_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ORGAN_ID  100 non-null    int64 
 1   项目名称      100 non-null    object
 2   财务账套名称    100 non-null    object
 3   level_1   100 non-null    object
 4   level_2   96 non-null     object
 5   level_3   61 non-null     object
 6   省         100 non-null    object
 7   市         100 non-null    object
 8   区         100 non-null    object
dtypes: int64(1), object(8)
memory usage: 7.2+ KB


In [62]:
df_project_full.set_index(['level_1', 'level_2', 'level_3'])

ORGAN_ID        项目名称            财务账套名称    省    市    区
level_1 level_2 level_3                                                       
产业园区事业部 产园-南京公司 仙东网谷      1424237        仙东网谷      镇江火炬置地发展有限公司  江苏省  镇江市  句容市
                合肥庐州意库    1427244      招商庐州意库     合肥德瀚产业园发展有限公司  安徽省  合肥市  庐阳区
                紫金智谷      1413199        紫金智谷     招商局地产（南京）有限公司  江苏省  南京市  秦淮区
                高铁网谷      1421251      招商高铁网谷      南京铁盛商业管理有限公司  江苏省  南京市  江宁区
                高铁网谷      1422197  招商高铁网谷（商业）      南京铁盛商业管理有限公司  江苏省  南京市  江宁区
...                           ...         ...               ...  ...  ...  ...
        番禺科技园   NaN       1421204   番禺创新科技园公寓    广州市番禺创新科技园有限公司  广东省  广州市  番禺区
        NaN     NaN       1413256        四海公园      深圳招商文化产业有限公司  广东省  深圳市  南山区
                NaN          1018         幼儿园       深圳市招商创业有限公司  广东省  深圳市  南山区
                NaN       1413264    索福德足球场土地  招商局蛇口工业区控股股份有限公司  广东省  深圳市  南山区
                NaN       1412259    网谷企业家俱乐部  招商局蛇口工业区控股股份有限公司  广东省  深圳市  南山区

[100 rows x 6 columns]